# Azure ML Compute Targets 

Тема лабораторной работы: работа с __Целевыми объектами вычислений__ (Compute Targets) в Azure ML.

## Соединение со Azure ML Workspace

Импорт необходимых модулей и проверка версии AzureML SDK:

In [ ]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

Устанавливаем соединение с Рабочей областью в Azure ML:

In [ ]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

## Создаем Целевой объект вычислений

Azure ML поддерживает ряд Целевых объектов вычислений, которые можно определить в своем Рабочем пространстве и использовать для выполнения Экспериментов. При этом оплата ресурсов производится только при их использовании.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'ml-cluster'

try:
    # First let's check that the cluster doesn't exist
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print(f'The cluster {cluster.name} already exists.')
except ComputeTargetException:
    # If the cluster doesn't exist, then create it
    try:
        cluster_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=1, max_nodes=3)
        cluster = ComputeTarget.create(ws, cluster_name, cluster_config)
        cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Проверим статус созданного Вычислительного кластера:

In [ ]:
cluster_state = cluster.get_status()

print(cluster_state.allocation_state, cluster_state.current_node_count, cluster_state.vm_size, cluster_state.creation_time)

## Запуск обучения ML модели на ML кластере

Запустим процесс обучения модели машинного обучения на только что созданном кластере.

### Подготовка

Используем [ранее загруженный](03-datastores-and-datasets.ipynb) датасет `diabets-db` в качестве данных для обучения: 

In [ ]:
data_ds = ws.datasets.get('diabetes_db')
print(f'Used dataset {data_ds.name}: {data_ds.description}')

Используем [ранее зарегистрированную](05A-environments.ipynb) Среду вычисления `diabetes-experiment-env` в качестве Environment для обучения модели:


In [ ]:
env = Environment.get(ws, 'diabetes-experiment-env')
print(f'Environment {env.name} will be used.')

Установим параметры Эксперимента:

In [ ]:
experiment_name = 'ml_cluster_demo'
experiment_dir = 'new_env_demo'
os.makedirs(experiment_dir, exist_ok=True)

Создадим Estimator, передав в качестве параметра `compute_target` имя созданного Вычислительного кластера:

In [ ]:
estimator = SKLearn(source_directory=experiment_dir,
                    inputs=[data_ds.as_named_input('data')],
                    entry_script='train-model.py',
                    script_params={'--reg_rate': 0.1},
                    compute_target=cluster_name, # Use the compute target created previously
                    environment_definition=env
                    )

Создаем Эксперимент и запускаем обучение модели:

In [ ]:
# Create and run the experiment
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=estimator)

# Get run details
run.wait_for_completion(show_output=True)
RunDetails(run).show()

Просмотрим результаты обучения модели:

In [ ]:
metrics = run.get_metrics()
for key in metrics.keys():
    print(key, metrics.get(key))
        
print('\n')
for file in run.get_file_names():
    print(file)

## Регистрация модели

Зарегистрируем обученную модель:

In [ ]:
run.register_model(model_path=run.get_file_names()[-1], model_name='diabetes_predict_model',
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']},
                   tags={'Demo':'Target compute'})

Получим список всех зарегистрированных ML моделей:

In [ ]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t', tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t', prop_name, ':', prop)